In [1]:
import pandas as pd
import os
import wiggum as wg
import numpy as np

We'll first load in some data, this has both regression and rate type trends, since this file has a weird index, we'll load it in as dataframe first

In [2]:
hit_search_rate = pd.read_csv('../data/state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI.csv',index_col='Unnamed: 0')
hit_search_rate.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0


We can now use the LabeledDataFrame with that DataFrame to create the object

In [3]:
labeled_df_setup = wg.LabeledDataFrame(hit_search_rate)

Next, we can infer the variable types

In [4]:
labeled_df_setup.infer_var_types()
labeled_df_setup.meta_df.head()

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,NaN,NaN,NaN
year,int64,ordinal,NaN,NaN,NaN
driver_gender,object,binary,NaN,NaN,NaN
driver_race,object,categorical,NaN,NaN,NaN
decriminalization,int64,binary,NaN,NaN,NaN


For this, we'll manually set these, but in the vizualization tool you can also set these with drop down menus

In [5]:
roles = {'state':['trend','groupby'], 'year':'trend', 'driver_gender':['trend','groupby'], 
         'driver_race':['trend','groupby'],
       'decriminalization':['groupby'], 'medical':['groupby'],
         'recreational':['groupby'], 'no_reforms':['groupby'],
       'search_conducted_false':'ignore', 'search_conducted_true':'ignore',
       'search_conducted_rate':'trend', 'contraband_found_false':'ignore',
       'contraband_found_true':'ignore', 'contraband_found_rate':'trend', 'hit_false':'ignore',
       'hit_true':'ignore', 'hit_rate':'trend', 'num_stops':'trend'}
is_count = {'state':False, 'year':False, 'driver_gender':False, 'driver_race':False,
       'decriminalization':False, 'medical':False, 'recreational':False, 'no_reforms':False,
       'search_conducted_false':True, 'search_conducted_true':True,
       'search_conducted_rate':False, 'contraband_found_false':True,
       'contraband_found_true':True, 'contraband_found_rate':False, 'hit_false':True,
       'hit_true':True, 'hit_rate':False, 'num_stops':True}
count_list = ['search_conducted_false', 'search_conducted_true','contraband_found_false',
       'contraband_found_true', 'hit_false',
       'hit_true', 'num_stops']
var_types = {'driver_gender':'categorical','decriminalization':'categorical', 
             'medical':'categorical', 'recreational':'categorical', 'no_reforms':False,}
weighting = {'hit_rate':'search_conducted_true','search_conducted_rate':'num_stops',
             'contraband_found_rate':'num_stops'}

We'll set those next.  Above gives examples of two ways that we can specify the count values to pass them to the set_counts function, but we'll only call it once below. 

In [6]:
labeled_df_setup.set_counts(count_list)
labeled_df_setup.set_roles(roles)
labeled_df_setup.set_weighting_vars(weighting)
labeled_df_setup.set_var_types(var_types)
labeled_df_setup.meta_df

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.meta_df['role'][k] = v
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/labeled_dataframe.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.meta_df['var_type'][k] = v


,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,trend,False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,ignore,True,NaN


Now, we've set this up, we can also save these configurations to load them in directly in the future

In [7]:
labeled_df_setup.to_csvs('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')

We can see what this does, using a bash magic

In [8]:
%%bash
cd ../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI
ls

df.csv
meta.csv
result_df.csv


it write the three DataFrames each out to their own .csv file in that directory. If that directory exists it will overwrite without warning, if not, also creates the directory. 

Now, we can can also load the data back

In [9]:
labeled_df = wg.LabeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN


In [10]:
labeled_df.add_intersectional(['driver_gender','driver_race'])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,search_conducted_rate,contraband_found_false,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,0.032258,31.0,NaN,NaN,1.0,NaN,NaN,31.0,F_White
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,0.666667,2.0,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,0.066667,15.0,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,0.111111,34.0,2.0,0.055556,2.0,2.0,0.5,36.0,M_White
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,0.049180,61.0,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian


We can also pass more variables and lengths of the tuples that we want it to combine.  For example if we provide 3 categorical variables we can do both pairs and triples of the variables.

In [11]:
labeled_df.add_intersectional(['driver_gender','driver_race','state'],[2,3])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race,driver_gender_state,driver_race_state,driver_gender_driver_race_state
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,NaN,NaN,1.0,NaN,NaN,31.0,F_White,F_AZ,White_AZ,F_White_AZ
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black,M_AZ,Black_AZ,M_Black_AZ
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic,M_AZ,Hispanic_AZ,M_Hispanic_AZ
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,0.055556,2.0,2.0,0.5,36.0,M_White,M_AZ,White_AZ,M_White_AZ
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian,F_AZ,Asian_AZ,F_Asian_AZ


In [12]:
labeled_df.add_all_dpgmm(qual_thresh =.2)
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,contraband_found_true,contraband_found_rate,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race,driver_gender_state,driver_race_state,driver_gender_driver_race_state
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,NaN,NaN,1.0,NaN,NaN,31.0,F_White,F_AZ,White_AZ,F_White_AZ
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,0.333333,1.0,1.0,0.5,3.0,M_Black,M_AZ,Black_AZ,M_Black_AZ
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,NaN,NaN,1.0,NaN,NaN,15.0,M_Hispanic,M_AZ,Hispanic_AZ,M_Hispanic_AZ
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,0.055556,2.0,2.0,0.5,36.0,M_White,M_AZ,White_AZ,M_White_AZ
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,NaN,NaN,3.0,NaN,NaN,61.0,F_Asian,F_AZ,Asian_AZ,F_Asian_AZ


In [13]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN
search_conducted_true,float64,continuous,[ignore],True,NaN


In [14]:
labeled_df.add_quantile(['hit_rate','num_stops'])
labeled_df.df.head()

,state,year,driver_gender,driver_race,decriminalization,medical,recreational,no_reforms,search_conducted_false,search_conducted_true,...,hit_false,hit_true,hit_rate,num_stops,driver_gender_driver_race,driver_gender_state,driver_race_state,driver_gender_driver_race_state,hit_ratequantiles,num_stopsquantiles
0,AZ,2009,F,White,0,0,0,1,30.0,1.0,...,1.0,NaN,NaN,31.0,F_White,F_AZ,White_AZ,F_White_AZ,high,low
1,AZ,2009,M,Black,0,0,0,1,1.0,2.0,...,1.0,1.0,0.5,3.0,M_Black,M_AZ,Black_AZ,M_Black_AZ,high,low
2,AZ,2009,M,Hispanic,0,0,0,1,14.0,1.0,...,1.0,NaN,NaN,15.0,M_Hispanic,M_AZ,Hispanic_AZ,M_Hispanic_AZ,high,low
3,AZ,2009,M,White,0,0,0,1,32.0,4.0,...,2.0,2.0,0.5,36.0,M_White,M_AZ,White_AZ,M_White_AZ,high,low
4,AZ,2010,F,Asian,0,1,0,0,58.0,3.0,...,3.0,NaN,NaN,61.0,F_Asian,F_AZ,Asian_AZ,F_Asian_AZ,high,low


In [15]:
labeled_df.get_vars_per_type('categorical')

['state',
 'driver_gender',
 'driver_race',
 'decriminalization',
 'medical',
 'recreational',
 'driver_gender_driver_race',
 'driver_gender_state',
 'driver_race_state',
 'driver_gender_driver_race_state',
 'hit_ratequantiles',
 'num_stopsquantiles']

In [16]:
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
state,object,categorical,"[trend, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,categorical,"[trend, groupby]",False,NaN
driver_race,object,categorical,"[trend, groupby]",False,NaN
decriminalization,int64,categorical,[groupby],False,NaN
medical,int64,categorical,[groupby],False,NaN
recreational,int64,categorical,[groupby],False,NaN
no_reforms,int64,False,[groupby],False,NaN
search_conducted_false,float64,continuous,[ignore],True,NaN
search_conducted_true,float64,continuous,[ignore],True,NaN


# Using Trends

Trend objects define their name, how to compute the trend and how to choose which variables, 

extension will allow that the var lists may be passed to reduce which ones are computed

In [17]:
corrobj = wg.All_Pearson()
corrobj.get_trend_vars(labeled_df)
corrobj.regression_vars

['year',
 'search_conducted_rate',
 'contraband_found_rate',
 'hit_rate',
 'num_stops']

In [18]:
rankobj = wg.Mean_Rank_Trend()
linreg_obj = wg.All_Linear_Trend()

# Computing Trends on a LabeledDataFrame

There are two ways, we can use default setting and pass the names of the trend type or a trend object

In [19]:
labeled_df.get_subgroup_trends_1lev(['pearson_corr'])
labeled_df.result_df.head()

,feat1,feat2,subgroup_trend,subgroup_trend_strength,group_feat,subgroup,trend_type,agg_trend,agg_trend_strength,comparison_type
0,year,search_conducted_rate,-0.247018,0.247018,state,AZ,pearson_corr,-0.03903,0.03903,aggregate-subgroup
1,year,search_conducted_rate,-0.414566,0.414566,state,CO,pearson_corr,-0.03903,0.03903,aggregate-subgroup
2,year,search_conducted_rate,0.118238,0.118238,state,CT,pearson_corr,-0.03903,0.03903,aggregate-subgroup
3,year,search_conducted_rate,-0.199765,0.199765,state,IL,pearson_corr,-0.03903,0.03903,aggregate-subgroup
4,year,search_conducted_rate,-0.603026,0.603026,state,MA,pearson_corr,-0.03903,0.03903,aggregate-subgroup


In [20]:
replace = False
labeled_df.result_df.empty  or replace

False

Now we can use a list of objects and apply multiple trends

In [21]:
labeled_df.get_subgroup_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/trend_components/regression.py:87: UserWarning: cannot compute with two different weights
  warnings.warn('cannot compute with two different weights')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/scipy/stats/_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:221: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:221: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/de

,feat1,feat2,subgroup_trend,subgroup_trend_strength,group_feat,subgroup,trend_type,agg_trend,agg_trend_strength,comparison_type
1872,contraband_found_rate,hit_rate,0.590074,0.590074,driver_gender_state,F_AZ,pearson_corr,0.275397,0.275397,aggregate-subgroup
957,year,num_stops,0.158044,0.158044,driver_gender_driver_race_state,F_Other_MD,pearson_corr,-0.043965,0.043965,aggregate-subgroup
1676,search_conducted_rate,num_stops,1,1.000000,driver_race_state,Other_CT,pearson_corr,-0.0655937,0.065594,aggregate-subgroup
2070,contraband_found_rate,hit_rate,0.738117,0.738117,driver_gender_driver_race_state,M_Other_NC,pearson_corr,0.275397,0.275397,aggregate-subgroup
365,year,contraband_found_rate,0.415297,0.415297,driver_race_state,Other_CO,pearson_corr,0.0411292,0.041129,aggregate-subgroup
4759,search_conducted_rate,state,[SC],NaN,driver_race_state,Black_SC,rank_trend,"[CO, NC, VT, WI, MD, MA, CT, TX, SC, WA, RI, I...",0.388100,aggregate-subgroup
2204,contraband_found_rate,num_stops,-0.0984349,0.098435,driver_race_state,Other_NC,pearson_corr,-0.0734563,0.073456,aggregate-subgroup
1939,contraband_found_rate,hit_rate,0.766563,0.766563,driver_race_state,Other_IL,pearson_corr,0.275397,0.275397,aggregate-subgroup
2239,contraband_found_rate,num_stops,-0.568132,0.568132,driver_gender_driver_race_state,F_Black_IL,pearson_corr,-0.0734563,0.073456,aggregate-subgroup
3310,contraband_found_rate,driver_gender,[F],NaN,driver_gender_driver_race_state,F_Other_TX,rank_trend,"[F, M]",0.179700,aggregate-subgroup


These two methods give the same, the string based version allows for simple access to default setting but passing a trend object would allow for overriding defaults and creating more custom subests of trends.

We can see what types of trends were computed from `result_df`

In [22]:
pd.unique(labeled_df.result_df['trend_type'])

array(['pearson_corr', 'rank_trend', 'lin_reg'], dtype=object)

In [23]:
pd.unique(labeled_df.result_df['comparison_type'])

array(['aggregate-subgroup'], dtype=object)

We can also add trends that are structured for pairwise comparisons

In [24]:
labeled_df.get_pairwise_trends_1lev([rankobj,linreg_obj])
labeled_df.result_df.sample(10)

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:313: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:313: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:313: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:313: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/

/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:313: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:313: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:313: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:313: RankWarning: Polyfit may be poorly conditioned
  curgroup_corr = cur_trend.get_trends(cur_grouping,'subgroup_trend')
/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/

appending


/home/smb/Documents/code/detect_simpsons_paradox_dev/env/lib/python3.6/site-packages/wiggum/detectors.py:341: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  self.result_df = pd.concat([self.result_df,pairwise_df],axis = 0)


,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,index,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type
21324,NaN,NaN,pairwise,year,contraband_found_rate,driver_gender_driver_race_state,332.0,F_Hispanic_SC,F_Hispanic_SC,-0.132787,-0.132787,0.000568,0.000568,lin_reg
13800,NaN,NaN,pairwise,hit_rate,driver_gender,driver_gender_driver_race_state,1339.0,M_Other_TX,M_Other_TX,[M],[M],NaN,NaN,rank_trend
3207,"[F, M]",0.1797,aggregate-subgroup,contraband_found_rate,driver_gender,driver_race_state,NaN,Asian_TX,NaN,"[F, M]",NaN,0.654700,NaN,rank_trend
24065,NaN,NaN,pairwise,search_conducted_rate,contraband_found_rate,driver_gender_driver_race_state,1017.0,M_Hispanic_WI,M_Hispanic_WI,0.00397331,0.00397331,0.000004,0.000004,lin_reg
8576,NaN,NaN,pairwise,num_stops,state,driver_gender_driver_race_state,33.0,F_Asian_CT,F_Asian_CT,[CT],[CT],NaN,NaN,rank_trend
5413,NaN,NaN,pairwise,num_stops,driver_gender,driver_race_state,10.0,Asian_AZ,Asian_AZ,"[F, M]","[F, M]",0.666700,0.666700,rank_trend
5474,NaN,NaN,pairwise,search_conducted_rate,driver_gender,driver_race_state,25.0,Asian_CT,Asian_CT,"[F, M]","[F, M]",1.000000,1.000000,rank_trend
3908,"[F, M]",0.0433,aggregate-subgroup,hit_rate,driver_gender,hit_ratequantiles,NaN,high,NaN,"[F, M]",NaN,0.000900,NaN,rank_trend
15215,NaN,NaN,pairwise,num_stops,search_conducted_rate,state,84.0,SC,SC,0.0229255,0.0229255,176.582777,176.582777,lin_reg
12336,NaN,NaN,pairwise,search_conducted_rate,driver_gender,driver_gender_driver_race_state,973.0,M_Black_MD,M_Black_MD,[M],[M],NaN,NaN,rank_trend


In [25]:
pd.unique(labeled_df.result_df['comparison_type'])

array(['aggregate-subgroup', 'pairwise'], dtype=object)

The object also stores the trend objects that have been applied, they can be used for mapping to get the distance functions that are appropriate for each trend

In [26]:
labeled_df.trend_list

In [27]:
labeled_df.result_df.head()

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,index,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type
0,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,state,NaN,AZ,NaN,-0.247018,NaN,0.247018,NaN,pearson_corr
1,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,state,NaN,CO,NaN,-0.414566,NaN,0.414566,NaN,pearson_corr
2,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,state,NaN,CT,NaN,0.118238,NaN,0.118238,NaN,pearson_corr
3,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,state,NaN,IL,NaN,-0.199765,NaN,0.199765,NaN,pearson_corr
4,-0.0390297,0.03903,aggregate-subgroup,year,search_conducted_rate,state,NaN,MA,NaN,-0.603026,NaN,0.603026,NaN,pearson_corr


In [28]:
# a = ['a','b','c','d']
# b = ['a','b','d','e']
a = ['F', 'M']
b = ['F', 'M']
for i, d in enumerate(set(a+b)):
    print (i,':',d)

0 : F
1 : M


In [29]:
# labeled_df.result_df['distance'] = labeled_df.result_df.apply(dist_helper,axis=1)
labeled_df.add_distance(row_wise=True) #('subgroup_trend','subgroup_trend2')

labeled_df.result_df.sample(10)

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,index,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance
3695,"[F, M]",0.043300,aggregate-subgroup,hit_rate,driver_gender,driver_gender_state,NaN,F_MD,NaN,[F],NaN,NaN,NaN,rank_trend,0.0
4369,"[F, M]",0.116400,aggregate-subgroup,num_stops,driver_gender,driver_gender_driver_race_state,NaN,M_Asian_TX,NaN,[M],NaN,NaN,NaN,rank_trend,NaN
5852,NaN,NaN,pairwise,search_conducted_rate,state,driver_race_state,120.0,Asian_WA,Asian_WA,[WA],[WA],NaN,NaN,rank_trend,NaN
1028,-0.043965,0.043965,aggregate-subgroup,year,num_stops,driver_gender_driver_race_state,NaN,M_Other_WI,NaN,-0.126186,NaN,0.126186,NaN,pearson_corr,0.0
23746,NaN,NaN,pairwise,search_conducted_rate,contraband_found_rate,driver_gender_driver_race_state,937.0,M_Hispanic_MA,M_Hispanic_MA,0.00274095,0.00274095,0.000070,0.000070,lin_reg,0.0
24443,NaN,NaN,pairwise,year,search_conducted_rate,driver_gender_driver_race_state,1111.0,M_Other_TX,M_Other_TX,4.46821,4.46821,0.015609,0.015609,lin_reg,0.0
3177,"[F, M]",0.179700,aggregate-subgroup,contraband_found_rate,driver_gender,driver_gender_state,NaN,F_MA,NaN,[F],NaN,NaN,NaN,rank_trend,0.0
10712,NaN,NaN,pairwise,contraband_found_rate,state,driver_gender_driver_race_state,567.0,F_Other_TX,F_Other_TX,[TX],[TX],NaN,NaN,rank_trend,NaN
6366,NaN,NaN,pairwise,hit_rate,driver_race,driver_race_state,248.0,Black_SC,Black_SC,[Black],[Black],NaN,NaN,rank_trend,NaN
2970,NaN,NaN,pairwise,num_stops,driver_race,state,87.0,VT,VT,"[Other, Hispanic, Asian, Black, White]","[Other, Hispanic, Asian, Black, White]",0.721400,0.721400,rank_trend,0.0


Each trend object has a trend_precompute dictionary as a property that stores the intermediate values (tables of the weighted rates for ranks and correlation matrices for pearson correlation, TODO: what do we need for linreg). These can be used in vizualization.

In [30]:
labeled_df.trend_list[0].trend_precompute

{'pearson_corr_agg_trend':                            year  search_conducted_rate  contraband_found_rate  \
 year                   1.000000              -0.039030               0.041129   
 search_conducted_rate -0.039030               1.000000               0.807915   
 contraband_found_rate  0.041129               0.807915               1.000000   
 hit_rate               0.316196              -0.094273               0.275397   
 num_stops             -0.043965              -0.065594              -0.073456   
 
                        hit_rate  num_stops  
 year                   0.316196  -0.043965  
 search_conducted_rate -0.094273  -0.065594  
 contraband_found_rate  0.275397  -0.073456  
 hit_rate               1.000000  -0.042102  
 num_stops             -0.042102   1.000000  ,
 'pearson_corr_subgroup_trend':                                               year  search_conducted_rate  \
 num_stopsquantiles                                                          
 high           

# Filtering

In [31]:
help(labeled_df.get_trend_rows)

Help on method get_trend_rows in module wiggum.ranking_processing:

get_trend_rows(feat1=None, feat2=None, group_feat=None, subgroup=None, trend_type=None) method of wiggum.labeled_dataframe.LabeledDataFrame instance
    return a row of result_df based on the specified values. returned rows
    meet provided criteria for all columns (and operator) and any one of the listed
    values for each column (or operator)
    
    Parameters
    -----------
    feat1 : str, list, or  None
        trend variable name or None to include all
    feat2 : str, list, or  None
        trend variable name or None to include all
    group_feat : str, list, or  None
        groupoby variable name or None to include all
    subgroup : str, list, or  None
        value of groupby_feat or or None to include all



So, we can use that function to filter and look at subsets of the trends based on the features, groupby, or subgroups

In [32]:
labeled_df.get_trend_rows(feat1='year',subgroup=['Black','Hispanic'])

44  total rows meet the criteria


,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,index,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance
16,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,driver_race,NaN,Black,NaN,-0.0391739,NaN,0.039174,NaN,pearson_corr,0.000000
17,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,driver_race,NaN,Hispanic,NaN,-0.177694,NaN,0.177694,NaN,pearson_corr,0.000000
278,0.0411292,0.041129,aggregate-subgroup,year,contraband_found_rate,driver_race,NaN,Black,NaN,0.0408099,NaN,0.040810,NaN,pearson_corr,0.000000
279,0.0411292,0.041129,aggregate-subgroup,year,contraband_found_rate,driver_race,NaN,Hispanic,NaN,0.0425516,NaN,0.042552,NaN,pearson_corr,0.000000
540,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_race,NaN,Black,NaN,0.354374,NaN,0.354374,NaN,pearson_corr,0.000000
541,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_race,NaN,Hispanic,NaN,0.329214,NaN,0.329214,NaN,pearson_corr,0.000000
802,-0.043965,0.043965,aggregate-subgroup,year,num_stops,driver_race,NaN,Black,NaN,-0.169689,NaN,0.169689,NaN,pearson_corr,0.000000
803,-0.043965,0.043965,aggregate-subgroup,year,num_stops,driver_race,NaN,Hispanic,NaN,0.014732,NaN,0.014732,NaN,pearson_corr,1.000000
16,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,driver_race,NaN,Black,NaN,-0.0391739,NaN,0.039174,NaN,pearson_corr,0.000000
17,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,driver_race,NaN,Hispanic,NaN,-0.177694,NaN,0.177694,NaN,pearson_corr,0.000000


In [33]:
labeled_df.get_trend_rows(group_feat = 'driver_race',trend_type ='lin_reg' )

75  total rows meet the criteria


,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,index,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance
5699,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,NaN,Asian,NaN,259.628,NaN,0.104919,NaN,lin_reg,0.997140
5700,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,NaN,Black,NaN,-2262.72,NaN,0.169689,NaN,lin_reg,0.000127
5701,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,NaN,Hispanic,NaN,422.667,NaN,0.014732,NaN,lin_reg,0.998086
5702,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,NaN,Other,NaN,91.5127,NaN,0.075180,NaN,lin_reg,0.992636
5703,-1560.86,0.043965,aggregate-subgroup,year,num_stops,driver_race,NaN,White,NaN,-5591.73,NaN,0.097383,NaN,lin_reg,0.000294
5961,1.96425,0.004473,aggregate-subgroup,year,search_conducted_rate,driver_race,NaN,Asian,NaN,1.24187,NaN,0.003783,NaN,lin_reg,0.131795
5962,1.96425,0.004473,aggregate-subgroup,year,search_conducted_rate,driver_race,NaN,Black,NaN,1.24021,NaN,0.001744,NaN,lin_reg,0.132212
5963,1.96425,0.004473,aggregate-subgroup,year,search_conducted_rate,driver_race,NaN,Hispanic,NaN,5.34096,NaN,0.017419,NaN,lin_reg,0.181954
5964,1.96425,0.004473,aggregate-subgroup,year,search_conducted_rate,driver_race,NaN,Other,NaN,-7.23536,NaN,0.032087,NaN,lin_reg,0.612781
5965,1.96425,0.004473,aggregate-subgroup,year,search_conducted_rate,driver_race,NaN,White,NaN,1.9543,NaN,0.003678,NaN,lin_reg,0.001309


We can also filter based on SP detections with `

In [34]:
labeled_df.get_SP_rows(thresh=.2)

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,index,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2
2,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,state,NaN,CT,NaN,0.118238,NaN,0.118238,NaN,pearson_corr,1.000000,True
8,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,state,NaN,SC,NaN,0.000440801,NaN,0.000441,NaN,pearson_corr,1.000000,True
12,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,state,NaN,WI,NaN,0.0485728,NaN,0.048573,NaN,pearson_corr,1.000000,True
13,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,driver_gender,NaN,F,NaN,0.0304108,NaN,0.030411,NaN,pearson_corr,1.000000,True
18,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,driver_race,NaN,Other,NaN,0.134696,NaN,0.134696,NaN,pearson_corr,1.000000,True
21,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,decriminalization,NaN,1,NaN,0.0780123,NaN,0.078012,NaN,pearson_corr,1.000000,True
26,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,no_reforms,NaN,0,NaN,0.0808758,NaN,0.080876,NaN,pearson_corr,1.000000,True
29,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,driver_gender_driver_race,NaN,F_Black,NaN,0.0681582,NaN,0.068158,NaN,pearson_corr,1.000000,True
31,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,driver_gender_driver_race,NaN,F_Other,NaN,0.137845,NaN,0.137845,NaN,pearson_corr,1.000000,True
36,-0.0390297,0.039030,aggregate-subgroup,year,search_conducted_rate,driver_gender_driver_race,NaN,M_Other,NaN,0.132174,NaN,0.132174,NaN,pearson_corr,1.000000,True


## Detection

Detection via `get_SP_rows` happens in two steps:
1. label the rows
2. filter by that column to return

Labeling the rows can happen in a number of ways too, the detection accepts a number of forms of input, custom detections can be built in many ways

In [35]:
help(labeled_df.label_SP_rows)

Help on method label_SP_rows in module wiggum.ranking_processing:

label_SP_rows(filter_thresh=None) method of wiggum.labeled_dataframe.LabeledDataFrame instance
    update the result_df with an additional colulmn indicateing rows with SP
    (or SP-like) as defined by sp_type
    
    Parameters
    -----------
    
    self : LabeledDataFrame
        must have values in result_df
    filter_thresh : dict or string
        dictionary of column label, threshold pairs or string name of a
        prespecified dictionary if dict, must include 'name' field (which
        will be used as the column name for storing the detections)



when filter_thresh is a dictionary, the filtering happens by taking the intersection of each row by the treshold prvided.  Some defaults are also built in accessible by string.

In [36]:
wg.trend_quality_sp

{'distance': 0.2,
 'agg_trend_strength': 0.15,
 'subgroup_trend_strength': 0.15,
 'name': 'default_qual_sp'}

Which can be applied with:

In [37]:
labeled_df.get_SP_rows('default_qual_sp')

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,index,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp
532,0.316196,0.316196,aggregate-subgroup,year,hit_rate,state,NaN,SC,NaN,-0.255096,NaN,0.255096,NaN,pearson_corr,1.0000,True,True
535,0.316196,0.316196,aggregate-subgroup,year,hit_rate,state,NaN,WA,NaN,-0.68403,NaN,0.684030,NaN,pearson_corr,1.0000,True,True
564,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_gender_state,NaN,F_CT,NaN,-0.262423,NaN,0.262423,NaN,pearson_corr,1.0000,True,True
570,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_gender_state,NaN,F_SC,NaN,-0.253289,NaN,0.253289,NaN,pearson_corr,1.0000,True,True
573,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_gender_state,NaN,F_WA,NaN,-0.646752,NaN,0.646752,NaN,pearson_corr,1.0000,True,True
575,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_gender_state,NaN,M_AZ,NaN,-0.15601,NaN,0.156010,NaN,pearson_corr,1.0000,True,True
583,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_gender_state,NaN,M_SC,NaN,-0.264436,NaN,0.264436,NaN,pearson_corr,1.0000,True,True
586,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_gender_state,NaN,M_WA,NaN,-0.768069,NaN,0.768069,NaN,pearson_corr,1.0000,True,True
598,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_race_state,NaN,Asian_WA,NaN,-0.723905,NaN,0.723905,NaN,pearson_corr,1.0000,True,True
599,0.316196,0.316196,aggregate-subgroup,year,hit_rate,driver_race_state,NaN,Asian_WI,NaN,-0.317996,NaN,0.317996,NaN,pearson_corr,1.0000,True,True


In [38]:
wg.DEFAULT_SP_DEF

{'distance': 0.0, 'name': 'SP', 'comparison_type': 'aggregate'}

Which can be applied with:

In [39]:
labeled_df.get_SP_rows('SP')

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,index,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp,SP


We can also define our own detection filters, using any available column

In [40]:
lin_only_qual = {'name':'lin_only_qual_sp','distance':.2, 'agg_trend_strength':.05,
                'subgroup_trend_strength':.15,'trend_type':'lin_reg'}
labeled_df.get_SP_rows(lin_only_qual,replace=True) 

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,index,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp,SP,lin_only_qual_sp


# Ranking

In [41]:
labeled_df.rank_occurences_by_view(ascending=False).head(20)

Index(['agg_trend', 'agg_trend_strength', 'comparison_type', 'feat1', 'feat2',
       'group_feat', 'index', 'subgroup', 'subgroup2', 'subgroup_trend',
       'subgroup_trend2', 'subgroup_trend_strength',
       'subgroup_trend_strength2', 'trend_type', 'distance', 'SP_thresh0.2',
       'default_qual_sp', 'SP', 'lin_only_qual_sp', 'mean_view_distance'],
      dtype='object')


,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,index,subgroup,subgroup2,subgroup_trend,subgroup_trend2,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp,SP,lin_only_qual_sp,mean_view_distance
20825,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_gender_driver_race,NaN,F_Black,NaN,[Black],NaN,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262
20828,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_gender_driver_race,NaN,F_White,NaN,[White],NaN,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262
20830,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_gender_driver_race,NaN,M_Black,NaN,[Black],NaN,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262
20833,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_gender_driver_race,NaN,M_White,NaN,[White],NaN,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262
20872,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_race_state,NaN,Black_AZ,NaN,[Black],NaN,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262
20873,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_race_state,NaN,Black_CO,NaN,[Black],NaN,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262
20874,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_race_state,NaN,Black_CT,NaN,[Black],NaN,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262
20875,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_race_state,NaN,Black_IL,NaN,[Black],NaN,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262
20876,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_race_state,NaN,Black_MA,NaN,[Black],NaN,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262
20877,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_race_state,NaN,Black_MD,NaN,[Black],NaN,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262


In [42]:
labeled_df.add_view_score('SP_thresh0.2',agg_type='sum',colored=False).head(10)

,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,index,subgroup,subgroup2,subgroup_trend,...,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp,SP,lin_only_qual_sp,mean_view_distance,sum_view_SP_thresh0.2
0,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_gender_driver_race,NaN,F_Black,NaN,[Black],...,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262,241.0
1,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_gender_driver_race,NaN,F_White,NaN,[White],...,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262,241.0
2,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_gender_driver_race,NaN,M_Black,NaN,[Black],...,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262,241.0
3,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_gender_driver_race,NaN,M_White,NaN,[White],...,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262,241.0
4,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_race_state,NaN,Black_AZ,NaN,[Black],...,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262,241.0
5,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_race_state,NaN,Black_CO,NaN,[Black],...,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262,241.0
6,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_race_state,NaN,Black_CT,NaN,[Black],...,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262,241.0
7,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_race_state,NaN,Black_IL,NaN,[Black],...,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262,241.0
8,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_race_state,NaN,Black_MA,NaN,[Black],...,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262,241.0
9,"[Other, Asian, Black, Hispanic, White]",0.4996,aggregate-subgroup,num_stops,driver_race,driver_race_state,NaN,Black_MD,NaN,[Black],...,NaN,NaN,rank_trend,0.8689,True,False,False,False,0.368262,241.0


In [43]:
labeled_df.rank_occurences_by_view('sum_view_SP_thresh0.2','SP_thresh0.2').head()

Index(['agg_trend', 'agg_trend_strength', 'comparison_type', 'feat1', 'feat2',
       'group_feat', 'index', 'subgroup', 'subgroup2', 'subgroup_trend',
       'subgroup_trend2', 'subgroup_trend_strength',
       'subgroup_trend_strength2', 'trend_type', 'distance', 'SP_thresh0.2',
       'default_qual_sp', 'SP', 'lin_only_qual_sp', 'mean_view_distance',
       'sum_view_SP_thresh0.2'],
      dtype='object')


,agg_trend,agg_trend_strength,comparison_type,feat1,feat2,group_feat,index,subgroup,subgroup2,subgroup_trend,...,subgroup_trend_strength,subgroup_trend_strength2,trend_type,distance,SP_thresh0.2,default_qual_sp,SP,lin_only_qual_sp,mean_view_distance,sum_view_SP_thresh0.2
13217,-0.043965,0.043965,aggregate-subgroup,year,num_stops,state,NaN,AZ,NaN,0.301916,...,0.301916,NaN,pearson_corr,1.0,True,False,False,False,0.169601,351.0
13218,-0.043965,0.043965,aggregate-subgroup,year,num_stops,state,NaN,MA,NaN,0.115809,...,0.115809,NaN,pearson_corr,1.0,True,False,False,False,0.169601,351.0
13219,-0.043965,0.043965,aggregate-subgroup,year,num_stops,state,NaN,NC,NaN,0.117245,...,0.117245,NaN,pearson_corr,1.0,True,False,False,False,0.169601,351.0
13220,-0.043965,0.043965,aggregate-subgroup,year,num_stops,state,NaN,RI,NaN,0.0583105,...,0.058311,NaN,pearson_corr,1.0,True,False,False,False,0.169601,351.0
13221,-0.043965,0.043965,aggregate-subgroup,year,num_stops,state,NaN,VT,NaN,0.0444106,...,0.044411,NaN,pearson_corr,1.0,True,False,False,False,0.169601,351.0
